# ML PROJECT

In [26]:
# Library imports and data upload 

In [3]:
import pandas as pd
import numpy as np
from sklearn import tree
from sklearn import naive_bayes
from sklearn import svm
from sklearn import linear_model
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
#from treeinterpreter import treeinterpreter as ti
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt

df=pd.read_csv('communities-crime-clean.csv')

In [28]:
#1. Decision Trees

In [29]:
##1.a. Create high-crime field

In [4]:
df['highCrime'] = (df['ViolentCrimesPerPop']>0.1).astype(int)  #.astype(int) to convert boolean into 0 and 1
pos = df.highCrime.mean()  #Percentage positive
neg = 1-df.highCrime.mean() #Percentage negative

In [31]:
#What are the percentage of positive and negative instances in the dataset?

In [5]:
print('Posistive= ',pos, '\n', "Negative= ", neg)

Posistive=  0.6271951831409934 
 Negative=  0.37280481685900657


In [33]:
##1.b. Use DecisionTreeClassifier to learn a decision tree to predict highCrime

In [6]:
inputX = df.drop(['highCrime','ViolentCrimesPerPop', 'communityname','fold'] ,axis=1,inplace=False)  #Input variables
outcomeY = df['highCrime']  #Outcome variable

treeModel = tree.DecisionTreeClassifier()  #Model to fit
treeFitted = treeModel.fit(inputX, outcomeY)#Estimated classifier
treePredicted = treeFitted.predict(inputX)  #Predicted values over the entire input dataset

In [35]:
###1.b.i. What are the training accuracy, precision, and recall

In [7]:
treeAcc = metrics.accuracy_score(outcomeY,treePredicted)  #Accuracy
treePre = metrics.precision_score(outcomeY, treePredicted)  #Precision
treeRec = metrics.recall_score(outcomeY, treePredicted)  #Recall

In [8]:
print(" Accuracy:",treeAcc, '\n',"Precision:",treePre,'\n',"Recall:",treeRec)

 Accuracy: 1.0 
 Precision: 1.0 
 Recall: 1.0


In [38]:
###1.b.ii. What are the main features used for classification

In [39]:
##################### NEEDS REVIEWING #####################

In [13]:
#import pydotplus
#dot_data = tree.export_graphviz(treeModel, out_file=None)
#graph = pydotplus.graph_from_dot_data(dot_data)
#graph.write_pdf("tree.pdf")

In [326]:
names = list(inputX.columns)
Feat = pd.DataFrame(treeFitted.feature_importances_, index = names, columns=['treef'])
print((Feat.sort_values(['treef'],axis=0, ascending = False))[0:10])

                     treef
PctKids2Par       0.358652
racePctWhite      0.092442
racePctHisp       0.047067
PctEmplManu       0.022134
HousVacant        0.017964
PctLess9thGrade   0.016859
PctEmploy         0.014602
racepctblack      0.012916
PctOccupMgmtProf  0.012818
MedRent           0.011718


In [43]:
##################### NEEDS REVIEWING #####################

In [44]:
##1.c.i. Now apply cross-validation to do 10-fold cross-validation

In [16]:
###Model evaluation with k-fold cross validation (k=10)
treeAcc_cv = cross_val_score(treeModel, inputX, outcomeY, cv=10, scoring='accuracy')
treePre_cv = cross_val_score(treeModel, inputX, outcomeY, cv=10, scoring='precision')
treeRec_cv = cross_val_score(treeModel, inputX, outcomeY, cv=10, scoring='recall')

In [46]:
##1.c.i. i.	What is the 10-fold cross-validation accuracy, precision, and recall for this method?

In [19]:
print(" Accuracy:",treeAcc_cv.mean(), '\n', "Precision:",treePre_cv.mean(),'\n',"Recall:",treeRec_cv.mean())

 Accuracy: 0.712974874372 
 Precision: 0.786051396449 
 Recall: 0.7712


In [48]:
##1.c.ii.	Why are they different from the results in the previous test?

In [49]:
#2.	Linear Classification

In [50]:
##2.a. Use GaussianNB to learn a Naive Bayes classifier to predict highCrime

In [20]:
nbModel = naive_bayes.GaussianNB ()  #Select GaussianNB as model
nbFit = nbModel.fit(inputX, outcomeY)  #Fit the GaussianNB with data
nbPredict = nbFit.predict(inputX)  #Prediction over the input model matrix

###Model evaluation with a single prediction over entire input
nbAcc = metrics.accuracy_score(outcomeY,nbPredict)  #Accuracy
nbPre = metrics.precision_score(outcomeY, nbPredict)  #Precision
nbRec = metrics.recall_score(outcomeY, nbPredict)  #Recall

nbAcc_cv = cross_val_score(nbModel, inputX, outcomeY, cv=10, scoring='accuracy')  #Accuracy over a 10-fold validation set
nbPre_cv = cross_val_score(nbModel, inputX, outcomeY, cv=10, scoring='precision')  #Precision over a 10-fold validation set
nbRec_cv = cross_val_score(nbModel, inputX, outcomeY, cv=10, scoring='recall')  #Recall over a 10-fold validation set

In [52]:
###2.a.i What is the 10-fold cross-validation accuracy, precision, and recall for this method?

In [21]:
print(" Accuracy:",nbAcc_cv.mean(), '\n',"Precision:",nbPre_cv.mean(),'\n',"Recall:",nbRec_cv.mean())

 Accuracy: 0.765108040201 
 Precision: 0.912419309104 
 Recall: 0.6976


In [54]:
###2.a.ii.

In [22]:
xMeans0 = inputX.groupby(nbPredict).mean().loc[0,:]  #Group mean for negative outcome
xMeans1 = inputX.groupby(nbPredict).mean().loc[1,:]  #Group mean for positive outcome

xStd0 = inputX.groupby(nbPredict).std().loc[0,:]  #Group standard deviation for negative outcome
xStd1 = inputX.groupby(nbPredict).std().loc[1,:]  #Group standard deviation for positive outcome

normAbsDiff = (xMeans1 - xMeans0).abs() / xStd0+xStd1  #Normalized absolute difference of groupwise (on outcome) means of all variables
normAbsDiffSorted = normAbsDiff.sort_values(ascending=False)  #Sort in descending order

In [56]:
###2.a.ii. What are the 10 most predictive features?

In [327]:
print(normAbsDiffSorted[0:10])  #Print 10 variables with the largest values

state               17.812025
NumIlleg             9.819252
NumStreet            4.498383
NumUnderPov          4.192369
PctIlleg             3.993768
racepctblack         3.988520
racePctWhite         3.941943
HousVacant           3.399270
NumImmig             3.336511
PctPersDenseHous     3.120286
dtype: float64


In [58]:
### Why do these make sense (or not)? 

In [59]:
###2.a.iii How do these results compare with your results from decision trees, above?

In [60]:
##2.b.	Use LinearSVC to learn a linear Support Vector Machine model

In [25]:
svmModel = svm.LinearSVC() #Select LinearSVC as model
svmFit = svmModel.fit(inputX, outcomeY)  #Fit the LinearSVC with data
svmPredict = svmFit.predict(inputX)  #Prediction over the input model matrix

###Model evaluation with a single prediction over entire input
svmAcc = metrics.accuracy_score(outcomeY,svmPredict)  #Accuracy
svmPre = metrics.precision_score(outcomeY, svmPredict)  #Precision
svmRec = metrics.recall_score(outcomeY, svmPredict)  #Recall

In [26]:
svmAcc_cv = cross_val_score(svmModel, inputX, outcomeY, cv=10, scoring='accuracy')  #Accuracy over a 10-fold validation set
svmPre_cv = cross_val_score(svmModel, inputX, outcomeY, cv=10, scoring='precision')  #Precision over a 10-fold validation set
svmRec_cv = cross_val_score(svmModel, inputX, outcomeY, cv=10, scoring='recall')  #Recall over a 10-fold validation set

In [63]:
###2.b.i Model evaluation with k-fold cross validation (where k=10)

In [27]:
print(" Accuracy:",svmAcc_cv.mean(), '\n',"Precision:",svmPre_cv.mean(),'\n',"Recall:",svmRec_cv.mean())

 Accuracy: 0.757185929648 
 Precision: 0.820500400199 
 Recall: 0.8808


In [65]:
###2.b.ii.	What are the 10 most predictive features?

In [28]:
xMeans0 = inputX.groupby(svmPredict).mean().loc[0,:]  #Group mean for negative outcome
xMeans1 = inputX.groupby(svmPredict).mean().loc[1,:]  #Group mean for positive outcome

xStd0 = inputX.groupby(svmPredict).std().loc[0,:]  #Group standard deviation for negative outcome
xStd1 = inputX.groupby(svmPredict).std().loc[1,:]  #Group standard deviation for positive outcome

normAbsDiff = (xMeans1 - xMeans0).abs() / xStd0+xStd1  #Normalized absolute difference of groupwise (on outcome) means of all variables
normAbsDiffSorted = normAbsDiff.sort_values(ascending=False)

In [29]:
print(normAbsDiffSorted[0:9])  #Print 10 variables with the largest values

state           17.812025
NumIlleg         9.819252
NumStreet        4.498383
NumUnderPov      4.192369
PctIlleg         3.993768
racepctblack     3.988520
racePctWhite     3.941943
HousVacant       3.399270
NumImmig         3.336511
dtype: float64


In [68]:
### Why do these make sense (or not)?

In [69]:
###2.b.iii.	How do these results compare with your results from decision trees, above?

In [ ]:
# Cleaning the 'full' data set

In [270]:
from sklearn.preprocessing import Imputer
#Load the full (dirty) dataset
df_full=pd.read_csv('communities-crime-full.csv')

##4.a. Create highCrime outcome variable
df_full['highCrime'] = (df_full['ViolentCrimesPerPop']>0.1).astype(int)  #.astype(int) to convert boolean into 0 and 1

#Drop variables that will not be used for classification
df_full_drop = df_full.drop(['communityname','fold','ViolentCrimesPerPop'],axis=1,inplace=False)  #Input variables

#Replace '?' with np.nan
df_full_drop.replace(to_replace='?', value=np.nan, inplace=True)
varlist = list(df_full_drop)

#Instantiate imputer for missing values using column median
impMedian = Imputer(missing_values='NaN', strategy='median', axis=0)

#Impute missing values
df_full_drop = pd.DataFrame(impMedian.fit_transform(df_full_drop))  #Replace with column median
df_full_drop.columns = varlist

#Define outcome vector and input matrix
inputX_full = df_full_drop.drop(['highCrime'],axis=1,inplace=False)  #Input variables
outcomeY_full = df_full_drop['highCrime']  #Outcome variable

In [71]:
#5.i. Experiment with two learning methods other than those described above

In [ ]:
# Method 1: Non-linear kernel for SVM classification

In [31]:
svmRBFModel = svm.SVC(kernel='rbf')  #Select non-linear SVC with RBF kernel
svmRBFFit = svmRBFModel.fit(inputX, outcomeY)  #Fit the model with data
svmRBFPredict = svmRBFFit.predict(inputX)  #Prediction over the input model matrix

In [ ]:
###Clean Data: Model evaluation with k-fold cross validation (where k=10)

In [ ]:
#MSE over a 10-fold validation set

In [33]:
svmRBFModelModelMSE_cv = cross_val_score(svmRBFModel, inputX, outcomeY, cv=10, scoring='neg_mean_squared_error')  

In [34]:
print(" MSE:",abs(svmRBFModelModelMSE_cv).mean())

 MSE: 0.416025125628


In [ ]:
#10-fold cross-validation accuracy, precision, and recall

In [35]:
svmRBFAcc_cv = cross_val_score(svmRBFModel, inputX, outcomeY, cv=10, scoring='accuracy')  #Accuracy over a 10-fold validation set
svmRBFPre_cv = cross_val_score(svmRBFModel, inputX, outcomeY, cv=10, scoring='precision')  #Precision over a 10-fold validation set
svmRBFRec_cv = cross_val_score(svmRBFModel, inputX, outcomeY, cv=10, scoring='recall')  #Recall over a 10-fold validation set

In [36]:
print(" Accuracy:",svmRBFAcc_cv.mean(), '\n',"Precision:",svmRBFPre_cv.mean(),'\n',"Recall:",svmRBFRec_cv.mean())

 Accuracy: 0.583974874372 
 Precision: 0.699992433194 
 Recall: 0.6504


In [ ]:
#>>> EDIT the following once 'full' dataset is added (notice 'inputX_full' and 'outcomeY_full' arguments below).

In [37]:
svmRBFModel_full = svm.SVC(kernel='rbf')  #Select non-linear SVC with RBF kernel
svmRBFFit_full = svmRBFModel_full.fit(inputX_full, outcomeY_full)  #Fit the model with data
svmRBFPredict_full = svmRBFFit_full.predict(inputX_full)  #Prediction over the input model matrix

In [ ]:
#MSE over a 10-fold validation set

In [38]:
svmRBFModelModelMSE_cv_full = cross_val_score(svmRBFModel_full, inputX_full, outcomeY_full, cv=10, scoring='neg_mean_squared_error')  

In [39]:
print("MSE:",abs(svmRBFModelModelMSE_cv_full).mean())

MSE: 0.288865671642


In [ ]:
###Full Data: Model evaluation with k-fold cross validation (where k=10)

In [40]:
svmRBFAcc_cv_full = cross_val_score(svmRBFModel_full, inputX_full, outcomeY_full, cv=10, scoring='accuracy')  #Accuracy over a 10-fold validation set
svmRBFPre_cv_full = cross_val_score(svmRBFModel_full, inputX_full, outcomeY_full, cv=10, scoring='precision')  #Precision over a 10-fold validation set
svmRBFRec_cv_full = cross_val_score(svmRBFModel_full, inputX_full, outcomeY_full, cv=10, scoring='recall')  #Recall over a 10-fold validation set

In [42]:
print(" Accuracy:",svmRBFAcc_cv_full.mean(), '\n',"Precision:",svmRBFPre_cv_full.mean(),'\n',"Recall:",svmRBFRec_cv_full.mean())

 Accuracy: 0.711134328358 
 Precision: 0.798245156148 
 Recall: 0.722596825397


In [72]:
## Method 2 Random Forrest

In [73]:
###Clean Data: Model evaluation with k-fold cross validation (where k=10)

In [43]:
RFModel = RandomForestClassifier()
RFFIt = RFModel.fit(inputX,outcomeY)
RFPredict = RFModel.predict(inputX)

In [ ]:
#MSE over a 10-fold validation set

In [44]:
RFModelMSE_cv = cross_val_score(RFModel, inputX, outcomeY, cv=10, scoring='neg_mean_squared_error')  

In [45]:
print("MSE:",abs(RFModelMSE_cv).mean())

MSE: 0.211801507538


In [ ]:
### 10-fold cross-validation accuracy, precision, and recall for Random Forrest with 'clean' data set

In [46]:
RFModelAcc_cv = cross_val_score(RFModel, inputX, outcomeY, cv=10, scoring='accuracy')  #Accuracy over a 10-fold validation set
RFModelPre_cv = cross_val_score(RFModel, inputX, outcomeY, cv=10, scoring='precision')  #Precision over a 10-fold validation set
RFModelRec_cv = cross_val_score(RFModel, inputX, outcomeY, cv=10, scoring='recall')  #Recall over a 10-fold validation set

In [47]:
print(" Accuracy:",RFModelAcc_cv.mean(), '\n',"Precision:",RFModelPre_cv.mean(),'\n',"Recall:",RFModelRec_cv.mean())

 Accuracy: 0.781685929648 
 Precision: 0.853263040205 
 Recall: 0.8096


In [77]:
#>>> EDIT the following once 'full' dataset is added (notice 'inputX_full' and 'outcomeY_full' arguments below).

In [48]:
RFModel_full = RandomForestClassifier()
RFFIt_full = RFModel_full.fit(inputX_full,outcomeY_full)
RFPredict_full = RFModel_full.predict(inputX_full)

In [49]:
RFModelMSE_cv_full = cross_val_score(RFModel_full, inputX_full, outcomeY_full, cv=10, scoring='neg_mean_squared_error')  
#MSE over a 10-fold validation set

In [51]:
print("MSE:",abs(RFModelMSE_cv_full).mean())

MSE: 0.175053513838


In [81]:
### 10-fold cross-validation accuracy, precision, and recall for Random Forrest with 'full' data set

In [52]:
RFModelAcc_cv_full = cross_val_score(RFModel_full, inputX_full, outcomeY_full, cv=10, scoring='accuracy')  #Accuracy over a 10-fold validation set
RFModelPre_cv_full = cross_val_score(RFModel_full, inputX_full, outcomeY_full, cv=10, scoring='precision')  #Precision over a 10-fold validation set
RFModelRec_cv_full = cross_val_score(RFModel_full, inputX_full, outcomeY_full, cv=10, scoring='recall')  #Recall over a 10-fold validation set

In [53]:
print(" Accuracy:",RFModelAcc_cv_full.mean(), '\n',"Precision:",RFModelPre_cv_full.mean(),'\n',"Recall:",RFModelRec_cv_full.mean())

 Accuracy: 0.816423798095 
 Precision: 0.867713446798 
 Recall: 0.833726984127


In [ ]:
#5.ii. What method gives the best results?

In [ ]:
#5.iii. What feature(s) seem to be most consistently predictive of high crime rates? How reliable is this conclusion?

In [ ]:
#6.c

In [276]:
df_new=pd.read_csv('industry.csv')
#df_new['communityname']=(df_new['communityname'].str.strip()).str.lower()
#df_full['communityname']=(df['communityname'].str.strip()).str.lower()

#df_new['communityname']=(df_new['communityname'].str.lower())
#df_full['communityname']=(df['communityname'].str.lower())

#df_full=df_full.replace(to_replace=np.nan,value='?')
df_full=df_full.replace(to_replace='?',value=np.nan)

df_new_sort=df_new.sort_values(['state','communityname'])
df_full_sort=df_full.sort_values(['state','communityname'])

In [274]:
df_new.head(5)

,state,community,Agriculture,Forestryandfisheries,Mining,Construction,Manufacturing,"Transportation,communications,andotherpublicutilities",Wholesaletrade,Retailtrade,"Finance,insurance,andrealestate",Businessandrepairservices,Personalservices,Entertainmentandrecreationservices,Professionalandrelatedservices,Publicadministration,StatePostalAbbreviation,communityname,COMN
0,1,820,0.003307,0.000000,0.007541,0.069454,0.126604,0.107554,0.065617,0.215372,0.084932,0.048948,0.016537,0.006879,0.219473,0.027781,AL,Alabastercity,Alabastercity
1,1,988,0.014788,0.000778,0.002024,0.062267,0.325031,0.047634,0.044988,0.199408,0.029421,0.030199,0.027397,0.009496,0.159869,0.046700,AL,Albertvillecity,Albertvillecity
2,1,1132,0.008782,0.000000,0.003572,0.036916,0.500298,0.020988,0.019202,0.114617,0.036618,0.027389,0.039446,0.010569,0.149300,0.032301,AL,AlexanderCitycity,AlexanderCitycity
3,1,1852,0.012198,0.000000,0.001052,0.036593,0.196425,0.056151,0.039012,0.206204,0.039537,0.025762,0.033123,0.012093,0.241430,0.100421,AL,Annistoncity,Annistoncity
4,1,2956,0.028664,0.000000,0.000646,0.085733,0.274112,0.059522,0.019755,0.164364,0.034603,0.034861,0.031246,0.004261,0.194061,0.068173,AL,Athenscity,Athenscity


Merge the new data set

We use an inner join to merge the two data sets using communityname and state. The reason for this is because some states might have the same communityname. We end up losing around 400 rows, which is considerable but those values where not present in the 'new' data set.

In [311]:
DF=df_full.merge(df_new,on=list(['communityname','state']),how='inner')

In [ ]:
#Divide outcome and input and replace missing values 

In [302]:
outcomeY_new=DF['highCrime']
inputX_new=DF.drop(['state','county','community_x','communityname','fold','StatePostalAbbreviation','COMN','ViolentCrimesPerPop','highCrime'],axis=1)

#Impute missing values
impMedian = Imputer(missing_values=np.nan, strategy='median', axis=0)
inputX_new= pd.DataFrame(impMedian.fit_transform(inputX_new))

In [ ]:
#Random Forrest "new" data set

In [303]:
RFModelAcc_cv_new = cross_val_score(RFModel_full, inputX_new, outcomeY_new, cv=10, scoring='accuracy')  #Accuracy over a 10-fold validation set
RFModelPre_cv_new = cross_val_score(RFModel_full, inputX_new, outcomeY_new, cv=10, scoring='precision')  #Precision over a 10-fold validation set
RFModelRec_cv_new = cross_val_score(RFModel_full, inputX_new, outcomeY_new, cv=10, scoring='recall')  #Recall over a 10-fold validation set

In [322]:
print(" New Data Set:",'\n', " Accuracy:",RFModelAcc_cv_new.mean(), '\n'," Precision:",RFModelPre_cv_new.mean(),'\n'," Recall:",RFModelRec_cv_new.mean())
print('\n',"Full Data Set:",'\n', " Accuracy:",RFModelAcc_cv_full.mean(), '\n'," Precision:",RFModelPre_cv_full.mean(),'\n'," Recall:",RFModelRec_cv_full.mean())

 New Data Set: 
  Accuracy: 0.818470496894 
  Precision: 0.875001302837 
  Recall: 0.882300884956

 Full Data Set: 
  Accuracy: 0.816423798095 
  Precision: 0.866949555561 
  Recall: 0.841695238095


In [ ]:
#LinearSVM "new" data set

In [305]:
svmModelAcc_cv_new = cross_val_score(svmModel, inputX_new, outcomeY_new, cv=10, scoring='accuracy')  #Accuracy over a 10-fold validation set
svmModelPre_cv_new = cross_val_score(svmModel, inputX_new, outcomeY_new, cv=10, scoring='precision')  #Precision over a 10-fold validation set
svmModelRec_cv_new = cross_val_score(svmModel, inputX_new, outcomeY_new, cv=10, scoring='recall')  #Recall over a 10-fold validation set

In [321]:
print(" New Data Set:",'\n', " Accuracy:",svmModelAcc_cv_new.mean(), '\n'," Precision:",svmModelPre_cv_new.mean(),'\n'," Recall:",svmModelRec_cv_new.mean())
print('\n',"Full Data Set:",'\n', " Accuracy:",svmAcc_cv.mean(), '\n'," Precision:",svmPre_cv.mean(),'\n'," Recall:",svmRec_cv.mean())

 New Data Set: 
  Accuracy: 0.613684006211 
  Precision: 0.695796878952 
  Recall: 0.581415929204

 Full Data Set: 
  Accuracy: 0.757185929648 
  Precision: 0.820500400199 
  Recall: 0.8808


In [ ]:
#Classification Tree "new" data set

In [308]:
treeModelAcc_cv_new = cross_val_score(treeModel, inputX_new, outcomeY_new, cv=10, scoring='accuracy')  #Accuracy over a 10-fold validation set
treeModelPre_cv_new = cross_val_score(treeModel, inputX_new, outcomeY_new, cv=10, scoring='precision')  #Precision over a 10-fold validation set
treeModelRec_cv_new = cross_val_score(treeModel, inputX_new, outcomeY_new, cv=10, scoring='recall')  #Recall over a 10-fold validation set

In [324]:
print(" New Data Set:",'\n', " Accuracy:",treeModelAcc_cv_new.mean(), '\n'," Precision:",treeModelPre_cv_new.mean(),'\n'," Recall:",treeModelRec_cv_new.mean())
print('\n',"Full Data Set:",'\n'," Accuracy:",treeAcc_cv_median.mean(), '\n', " Precision:",treePre_cv_median.mean(),'\n'," Recall:",treeRec_cv_median.mean())

 New Data Set: 
  Accuracy: 0.763552018634 
  Precision: 0.836555679578 
  Recall: 0.828318584071

 Full Data Set: 
  Accuracy: 0.712974874372 
  Precision: 0.786051396449 
  Recall: 0.7712
